import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [74]:
nations = np.genfromtxt("nations.csv", delimiter=",", skip_header=True) 

In [79]:
"""
Problem definition:

Use the new data set for this exercise cars.csv. 
The data can be read with the command cars = np.genfromtxt("cars.csv", delimiter=",", skip_header=True). In this exercise, you do not try to inference one column from another, but rather use the first 6 columns to predict the 7th column. The aim is to predict the fuel consumption of cars.

The meaning of the columns in the cars matrix:
1. "cylinders" = number of the cylinders
2. "displacement" = space in the back of the car
3. "horsepower" = power of the engine
4. "weight" = weight in pounds
5. "acceleration" = time to accelerate from 0 to 60 miles per hour
6. "model year" = release date
7. "mpg" = miles per gallon

First normalize all data, use the first 6 columns as input data and try to predict the 7th column. Evaluate your predictions with the RMSE. Since several input data/columns are now used, please replace the polynomial function with a linear combination of weighted input values. Try to find some random coefficients which create a good approximation function, just to get a feeling for the data.

Polynomial Function:
polynomial function

Weighted Function:
weighted function


Exchange your random-picking algorithm with the evolution strategy. Implement a way to define the number of offspring and generations. Make it possible to keep the parents (generation) if they are better than their children.

All children are initially a copy of their parents np.tile(...) but are modified a little bit for each coefficient. At the end of an epoch, the best descendant is determined (from the set of all children and/or parents). and used as the parent of the next epoch.

Let your best combination of coefficients predict the fuel consumption for the following four lines:

initial RMSE: 21.56
Run 300 rounds ES(children:3 parents:1) ...
best RMSE: 4.38
Line 4: mpg is 16.00, prediction was 14.70
Line 57: mpg is 24.00, prediction was 23.27
Line 117: mpg is 29.00, prediction was 27.82
Line 219: mpg is 33.50, prediction was 30.36
"""

'\nProblem definition:\n\nUse the new data set for this exercise cars.csv. \nThe data can be read with the command cars = np.genfromtxt("cars.csv", delimiter=",", skip_header=True). In this exercise, you do not try to inference one column from another, but rather use the first 6 columns to predict the 7th column. The aim is to predict the fuel consumption of cars.\n\nThe meaning of the columns in the cars matrix:\n1. "cylinders" = number of the cylinders\n2. "displacement" = space in the back of the car\n3. "horsepower" = power of the engine\n4. "weight" = weight in pounds\n5. "acceleration" = time to accelerate from 0 to 60 miles per hour\n6. "model year" = release date\n7. "mpg" = miles per gallon\n\nFirst normalize all data, use the first 6 columns as input data and try to predict the 7th column. Evaluate your predictions with the RMSE. Since several input data/columns are now used, please replace the polynomial function with a linear combination of weighted input values. Try to fin

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

cars = np.genfromtxt("cars.csv", delimiter=",", skip_header=True)

# Funktion zur Berechnung des root mean square errors
def getRMSE (werte, erwarteteWerte):
    quadratischerFehlerSum =  np.sum(np.power(erwarteteWerte - werte, 2))
    mse = quadratischerFehlerSum / len(werte)
    rmse = np.sqrt(mse)
    return rmse


# normalisierung
def normalize(x, xMin, xMax):
    y = (x - xMin)/ (xMax - xMin)
    return y

# denormalisierung
def denormalize(xNorm, xMin, xMax):
    y =  xNorm * (xMax - xMin) + xMin
    return y

#
def getWeighted(xValues, koeffizienten):
    y = 0
    for i in range (len(koeffizienten)):
        y = y + xValues[i] * koeffizienten[i]
    return y

#Spalte started bei 1, da 0 die Modellbezeichnung ist, Maximalwert für Normalisierung speichern          
cylinders = cars[:,1]    
displacement = cars[:,2]  
horsepower = cars[:,3]  
weight = cars[:,4]  
acceleration = cars[:,5]   
modelYear = cars[:,6]  
mpg = cars[:,7]                         
            
#Maximum der Features bestimmen für Normalisieren
cylindersMax = np.amax(cylinders)    
displacementMax = np.amax(displacement)  
horsepowerMax = np.amax(horsepower)  
weightMax = np.amax(weight)  
accelerationMax = np.amax(acceleration)   
modelYearMax = np.amax(modelYear)  
mpgMax = np.max(mpg)    

#Minimum der Features bestimmen für Normalisieren
cylindersMin = np.amin(cylinders)    
displacementMin = np.amin(displacement)  
horsepowerMin = np.amin(horsepower)  
weightMin = np.amin(weight)  
accelerationMin = np.amin(acceleration)   
modelYearMin = np.amin(modelYear)  
mpgMin = np.amin(mpg)  
                    
#normalisieren aller Werte der Spaltenattribute
cylindersNorm = normalize(cylinders, cylindersMin, cylindersMax)
displacementNorm = normalize(displacement, displacementMin, displacementMax)
horsepowerNorm = normalize(horsepower, horsepowerMin, horsepowerMax)
weightNorm = normalize(weight, weightMin, weightMax)
accelerationNorm = normalize(acceleration, accelerationMin, accelerationMax)
modelYearNorm = normalize(modelYear, modelYearMin, modelYearMax)
mpgNorm = normalize(mpg, mpgMin, mpgMax)

#spaltenvektoren
cylindersVector = np.expand_dims(cylindersNorm, axis=1)
displacementVector = np.expand_dims(displacementNorm, axis=1)
horsepowerVector = np.expand_dims(horsepowerNorm, axis=1)
weightVector = np.expand_dims(weightNorm, axis=1)
accelerationVector = np.expand_dims(accelerationNorm, axis=1)
modelYearVector = np.expand_dims(modelYearNorm, axis=1)

                
#wertematrix 
werteMatrix = np.column_stack([cylindersVector, displacementVector, horsepowerVector, weightVector, accelerationVector, modelYearVector])


koeffizientenElter = np.expand_dims(np.array([random.uniform(-1.0, 1.0), random.uniform(-1.0, 1.0), 
                                            random.uniform(-1.0, 1.0), random.uniform(-1.0, 1.0), 
                                            random.uniform(-1.0, 1.0), random.uniform(-1.0, 1.0)]), axis = 1)

besteKindKoeffizienten = koeffizientenElter

mpgPredictionNormElter = np.squeeze(np.matmul(werteMatrix, koeffizientenElter))

initialRMSE = getRMSE (denormalize(mpgNorm, mpgMin, mpgMax), denormalize(mpgPredictionNormElter, mpgMin, mpgMax))

smallestRMSEValue = float("inf")

#loop über n epochen
epochen = 300
for i in range(0, epochen): 
    kinderanzahl = 3

    for i in range(kinderanzahl): 
        mutatorKoeffizientenkind = np.expand_dims(np.array([random.uniform(-1.0, 1.0), random.uniform(-1.0, 1.0), 
                                                random.uniform(-1.0, 1.0), random.uniform(-1.0, 1.0), 
                                                random.uniform(-1.0, 1.0), random.uniform(-1.0, 1.0)]), axis = 1)
    
        koeffizientenKind = besteKindKoeffizienten + mutatorKoeffizientenkind / 10

        #mpg Erwartungswerte ausrechnen & RMSE bestimmen
        mpgPredictionNormKind = np.squeeze(np.matmul(werteMatrix, koeffizientenKind))
        
        rmseKind = getRMSE (denormalize(mpgNorm, mpgMin, mpgMax), denormalize(mpgPredictionNormKind, mpgMin, mpgMax))

        if(rmseKind < smallestRMSEValue):
                smallestRMSEValue = rmseKind
                besteKindKoeffizienten = koeffizientenKind
                #print("smallest RMSE Value: {} ".format(smallestRMSEValue))

print(np.shape(besteKindKoeffizienten))
print("initial RMSE: {} ".format(initialRMSE))
print("smallest RMSE Value: {} ".format(smallestRMSEValue))
print("Line 4: mpg is {} prediction was: {}  ".format(cars[3,7], denormalize(getWeighted(werteMatrix[3, 0:6], besteKindKoeffizienten), mpgMin, mpgMax)))
print("Line 57: mpg is {} prediction was: {}  ".format(cars[56,7],denormalize(getWeighted(werteMatrix[56, 0:6], besteKindKoeffizienten), mpgMin, mpgMax)))
print("Line 117: mpg is {} prediction was: {}  ".format(cars[116,7],denormalize(getWeighted(werteMatrix[116, 0:6], besteKindKoeffizienten), mpgMin, mpgMax)))
print("Line 219: mpg is {} prediction was: {}  ".format(cars[218,7],denormalize(getWeighted(werteMatrix[218, 0:6], besteKindKoeffizienten), mpgMin, mpgMax)))



(6, 1)
initial RMSE: 47.36250866530023 
smallest RMSE Value: 4.418028394547978 
Line 4: mpg is 16.0 prediction was: [15.21709313]  
Line 57: mpg is 24.0 prediction was: [24.19960175]  
Line 117: mpg is 29.0 prediction was: [28.88724432]  
Line 219: mpg is 33.5 prediction was: [31.44062758]  
